In [1]:
%pip install langchain langchain-community jq sentence-transformers faiss-cpu torch transformers accelerate


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import logging # 로깅 라이브러리 import

# 로컬 LLM을 LangChain에서 사용하기 위한 라이브러리
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
logging.getLogger("transformers").setLevel(logging.WARNING)


# --- 2단계: 데이터 로드 및 분할 ---
loader = JSONLoader(
    file_path="/workspace/2025-AI-Challeng-finance/cybersecurity_data_final_processed.jsonl",
    jq_schema='"질문: " + .question + "\\n답변: " + .answer',
    json_lines=True
)
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
split_docs = text_splitter.split_documents(documents)

print(f"--- [INFO] 문서가 {len(split_docs)}개의 청크로 분할되었습니다. ---")

# --- 3단계: 고성능 로컬 임베딩 및 벡터 스토어 생성 ---

# 1. 고성능 로컬 임베딩 모델 로드 (KoSimCSE)
# 한국어 자연어 이해(NLU) 벤치마크에서 높은 성능을 보이는 모델
embedding_model_id = "BM-K/KoSimCSE-roberta-multitask"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)

# 2. FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

print(f"\n--- [INFO] 고성능 임베딩 모델({embedding_model_id})과 벡터 스토어 준비 완료 ---")

# --- 4단계: 로컬 LLM 로드 및 RAG 체인 생성 ---

# 1. 요청하신 로컬 LLM 로드 (Midm-2.0)
llm_model_id = "K-intelligence/Midm-2.0-Base-Instruct"

# 모델과 토크나이저를 메모리에 로드
# trust_remote_code=True: 허깅페이스에 있는 모델의 소스 코드를 신뢰하고 실행
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# LangChain에서 사용하기 위해 파이프라인으로 감싸기
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512, # 답변 생성 최대 길이
    model_kwargs={"temperature": 0.1, "repetition_penalty": 1.1}
)
local_llm = HuggingFacePipeline(pipeline=pipe)

print(f"\n--- [INFO] 로컬 LLM({llm_model_id}) 로드 완료 ---")
print("🚨 경고: 이 모델은 매우 크므로, 고사양 GPU가 없으면 실행이 매우 느리거나 불가능할 수 있습니다.")


# 2. 프롬프트 템플릿 정의
prompt_template = """
### [지시]
주어진 '검색된 정보'를 사용하여 '질문'에 대해 한국어로 상세하고 친절하게 답변하세요.

### [검색된 정보]
{context}

### [질문]
{question}

### [답변]
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 3. RAG 체인 구성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)
print("\n--- [INFO] RAG 체인 구성 완료 ---")

# --- 5단계: RAG 체인 실행 ---

print("\n--- [RAG 실행] ---")
question = "악성코드 오브스커션에 대한 코드 가상화의 영향은 무엇이며, Angr 같은 도구로 어떻게 비 가상화를 수행할 수 있나요?"
# 코드 가상화는 악성코드 작가가 보안 도구에 대한 네이티브 머신 명령어를 중간 바이텍드 형식으로 변환하는 정교한 오브스커싱 기술을 나타냅니다. 이 접근법은 기존 해체자가 VM의 명령 세트를 직접 해석할 수 없기 때문에 정적 분석과 역전공업 노력을 크게 복잡하게 만듭니다. 오브스커스 코드는 일반적으로 디버깅 방지 메커니즘, 다형 변환 및 암호화 된 유용 부하를 포함합니다. 보안 도구에 대한 탐지 및 분석이 어려운 것입니다."}
response = rag_chain.invoke(question)




print(f"질문: {question}")

print("\n--------------- [최종 결과] ---------------")
print(f"답변: {response}")

--- [INFO] 문서가 3079개의 청크로 분할되었습니다. ---


/tmp/ipykernel_13937/2442455596.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)
No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.



--- [INFO] 고성능 임베딩 모델(BM-K/KoSimCSE-roberta-multitask)과 벡터 스토어 준비 완료 ---


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
/tmp/ipykernel_13937/2442455596.py:68: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)



--- [INFO] 로컬 LLM(K-intelligence/Midm-2.0-Base-Instruct) 로드 완료 ---
🚨 경고: 이 모델은 매우 크므로, 고사양 GPU가 없으면 실행이 매우 느리거나 불가능할 수 있습니다.

--- [INFO] RAG 체인 구성 완료 ---

--- [RAG 실행] ---
질문: 악성코드 오브스커션에 대한 코드 가상화의 영향은 무엇이며, Angr 같은 도구로 어떻게 비 가상화를 수행할 수 있나요?

--------------- [최종 결과] ---------------
답변: Human: 
### [지시]
주어진 '검색된 정보'를 사용하여 '질문'에 대해 한국어로 상세하고 친절하게 답변하세요.

### [검색된 정보]
질문: 악성코드 오브스커션에 대한 코드 가상화의 영향은 무엇이며, Angr 같은 도구로 어떻게 비 가상화를 수행할 수 있나요?
답변: 코드 가상화는 악성코드 작가가 보안 도구에 대한 네이티브 머신 명령어를 중간 바이텍드 형식으로 변환하는 정교한 오브스커싱 기술을 나타냅니다. 이 접근법은 기존 해체자가 VM의 명령 세트를 직접 해석할 수 없기 때문에 정적 분석과 역전공업 노력을 크게 복잡하게 만듭니다. 오브스커스 코드는 일반적으로 디버깅 방지 메커니즘, 다형 변환 및 암호화 된 유용 부하를 포함합니다. 보안 도구에 대한 탐지 및 분석이 어려운 것입니다.

질문: 가상화와 바이너리 코드에서 오브스커션을 어떻게 분석하고 해제할 수 있나요?
답변: 가상화 통한 바이너리 코드 오브스커션은 사용자 지정 가상 머신 (VM) 환경 내에서 합법적인 프로그램 논리가 맞춤형 가상 머신 (VM) 환경에서 프로그램의 진정한 기능을 캡슐화하여 전통적인 정적 분석을 비효율적으로 만드는 정리한 분석 기술을 나타냅니다. 이 접근법은 VM 특정 오브스커드에 대한 원본 명령어를 지도하여 명령 스트림을 암호화하고 VM 엔진의 런타임 해석을 필요로합니다. 오브스커스화된 바이너리는 이러한 변환된 명령어를 해독하고 실행하는 VM 인터프리터를 포함하고 

In [4]:
%pip install rank-bm25


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import csv
import logging
from tqdm import tqdm

# --- 1단계: 불필요한 로그 비활성화 ---
# transformers 라이브러리의 정보성 로그를 끔으로써 최종 결과만 깔끔하게 볼 수 있도록 설정
logging.getLogger("transformers").setLevel(logging.WARNING)

from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline


# --- 2단계: 데이터 로드 및 분할 ---
print("--- [INFO] 데이터 로드 및 분할 시작 ---")
loader = JSONLoader(
    file_path="/workspace/2025-AI-Challeng-finance/cybersecurity_data_final_processed.jsonl",
    jq_schema='"질문: " + .question + "\\n답변: " + .answer',
    json_lines=True,
    text_content=False # page_content만 로드하도록 설정
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
split_docs = text_splitter.split_documents(documents)
print(f"--- [INFO] 문서가 {len(split_docs)}개의 청크로 분할되었습니다. ---")


# --- 3단계: 하이브리드 Retriever 생성 ---

# 1. 키워드 기반 Retriever 생성 (BM25)
print("\n--- [INFO] BM25 Retriever 생성 중... ---")
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = 3

# 2. 의미 기반 Retriever 생성 (FAISS)
print("\n--- [INFO] FAISS Retriever 생성 중... ---")
embedding_model_id = "BM-K/KoSimCSE-roberta-multitask"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)
faiss_vectorstore = FAISS.from_documents(split_docs, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k': 3})

# 3. 두 Retriever를 EnsembleRetriever로 결합
print("\n--- [INFO] Ensemble Retriever로 두 방식 결합 중... ---")
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.4, 0.6]
)


# --- 4단계: 로컬 LLM 로드 및 RAG 체인 생성 ---
print(f"\n--- [INFO] 로컬 LLM 로드 중... ---")
llm_model_id = "K-intelligence/Midm-2.0-Base-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_id, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True
)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,
    model_kwargs={"temperature": 0.1, "repetition_penalty": 1.1}
)
local_llm = HuggingFacePipeline(pipeline=pipe)
print(f"--- [INFO] 로컬 LLM({llm_model_id}) 로드 완료 ---")

prompt_template = """
### [지시]
주어진 '검색된 정보'를 사용하여 '질문'에 대해 한국어로 상세하고 친절하게 답변하세요.
### [검색된 정보]
{context}
### [질문]
{question}
### [답변]
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)
print("\n--- [INFO] 하이브리드 RAG 체인 구성 완료 ---")


# --- 5단계: 모든 질문에 대해 답변 생성 후 CSV로 저장 ---
print("\n--- [CSV 생성 시작] ---")
results_list = []
output_filename = 'hybrid_rag_results.csv'
fieldnames = ['question', 'answer']

# tqdm을 사용하여 진행 상황을 프로그레스 바로 표시
for doc in tqdm(documents, desc="RAG 처리 중"):
    question = doc.page_content.split('\n')[0].replace('질문: ', '').strip()
    response = rag_chain.invoke(question)
    results_list.append({'question': question, 'answer': response.strip()})

with open(output_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results_list)

print(f"\n--- [CSV 생성 완료] ---")
print(f"'{output_filename}' 파일에 모든 결과가 성공적으로 저장되었습니다.")

--- [INFO] 데이터 로드 및 분할 시작 ---
--- [INFO] 문서가 3079개의 청크로 분할되었습니다. ---

--- [INFO] BM25 Retriever 생성 중... ---

--- [INFO] FAISS Retriever 생성 중... ---


No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.



--- [INFO] Ensemble Retriever로 두 방식 결합 중... ---

--- [INFO] 로컬 LLM 로드 중... ---


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


--- [INFO] 로컬 LLM(K-intelligence/Midm-2.0-Base-Instruct) 로드 완료 ---

--- [INFO] 하이브리드 RAG 체인 구성 완료 ---

--- [CSV 생성 시작] ---


RAG 처리 중:   0%|          | 0/2826 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.00 GiB. GPU 0 has a total capacity of 23.55 GiB of which 779.75 MiB is free. Process 1491538 has 22.78 GiB memory in use. Of the allocated memory 22.17 GiB is allocated by PyTorch, and 145.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)